## コードリーディング

>TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。
>
>tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。



## 【問題1】スクラッチを振り返る
>ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。
>
>（例）
>
>- 重みを初期化する必要があった
>- エポックのループが必要だった
>
>それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

#### 解答
- 重みとバイアスを初期化する必要があった
- 全結合層のクラスを用意する必要があった
- 活性化関数のクラスを用意する必要があった
- 全結合層や活性化関数のクラスに順伝播と逆伝播のメソッドを用意する必要があった
- 最適化手法クラスに順伝播と逆伝播のメソッドを用意する必要があった
- 全結合層の入力サイズと出力サイズを考慮する必要があった
- エポックのループが必要だった
- 逆伝播の最後に重みとバイアスの更新が必要だった
- 損失関数を計算して損失グラフが適切に下がっているか確認が必要だった

## 【問題2】スクラッチとTensorFlowの対応を考える
>以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。
>
>それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。
>
>《サンプルコード》
>
>＊バージョン1.5から1.14の間で動作を確認済みです。

#### 解答
- 重みとバイアスを初期化する必要があった
    - weightsとbiasesのなかにVariableを用意し、logitsへ代入
- 全結合層のクラスを用意する必要があった
    - 全体構造を決めるexample_net()関数を作成し、logitsへ代入
- 活性化関数のクラスを用意する必要があった
    - layer_1 = tf.nn.relu(layer_1)
- 全結合層や活性化関数のクラスに順伝播と逆伝播のメソッドを用意する必要があった
    - logits = example_net(X)でネットワーク構造を指定すればTensorFlowが実行してくれる
- 最適化手法クラスに順伝播と逆伝播のメソッドを用意する必要があった
    - 最適化手法optimizer変数にAdamOptimizerという最適化メソッドを代入して使用
- 全結合層の入力サイズと出力サイズを考慮する必要があった
    - 変数で指定して重みとバイアスの宣言で使用
- エポックのループが必要だった
    - 変数で指定してfor分でループ（スクラッチでも同様）
- 逆伝播の最後に重みとバイアスの更新が必要だった
    - 更新式を用意 : layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    - logitsへ代入
    - 最適化インスタンスの引数に渡す。loss_op に目的関数を格納する際、logitsを引数として与えることでAdamOptimizerにて更新される。
- 損失関数を計算して損失グラフが適切に下がっているか確認が必要だった
    - 変数（loss_op、Accuracy）に数式を入れてTensorFlowに渡し、
    実行時に計算させる。

### データセットの用意
>以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。
>
>Iris Species
>
>目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。
>
>- Iris-versicolor
>- Iris-virginica

In [1]:
import pandas as pd
# scikit-learn から　iris データセットを読み込む
from sklearn.datasets import load_iris

# iris データセットを変数に格納する
data = load_iris()

# 行と列のインデックス用のリストを用意
iris_data_columns=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
iris_target_columns=['Species',]
# pandas のデータフレームにする
X_data = pd.DataFrame(data=data['data'], columns=iris_data_columns)
y_data = pd.DataFrame(data=data['target'], columns=iris_target_columns)

y_data=y_data.replace({0: 'Iris-setosa', 1: 'Iris-versicolor', 2:'Iris-virginica'})

# Xとyを列方向に結合
df = pd.concat([X_data, y_data], axis=1)
display(df)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# データセットの読み込み
#dataset_path ="Iris.csv"
#df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df_q2 = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df_q2["Species"]
X = df_q2.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
    
    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)# variableの初期化
    for epoch in range(num_epochs):
        # エポックごとにループ np.ceil=小数の切り上げと切り下げ（切り捨て）
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #train_op = AdamOptimizer Adamアルゴリズムによる勾配降下法
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
non-resource variables are not supported in the long term
Epoch 0, loss : 33.4553, val_loss : 17.2785, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 4.5864, val_loss : 7.3239, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 0.4350, val_loss : 1.2601, acc : 0.750, val_acc : 0.750
Epoch 3, loss : 0.1076, val_loss : 1.2723, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 0.5112, val_loss : 2.2364, acc : 0.750, val_acc : 0.562
Epoch 5, loss : 1.1868, val_loss : 2.6200, acc : 0.750, val_acc : 0.750
Epoch 6, loss : 1.0769, val_loss : 2.6816, acc : 0.750, val_acc : 0.625
Epoch 7, loss : 0.9539, val_loss : 2.5139, acc : 0.750, val_acc : 0.750
Epoch 8, loss : 1.8821, val_loss : 4.1873, acc : 0.750, val_acc : 0.500
Epoch 9, loss : 1.3323, val_loss : 2.7285, acc : 0.750, val_acc : 0.750
test_acc : 0.750


## 他のデータセットへの適用
>これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。
>
>- Iris（3種類全ての目的変数を使用）
>- House Prices
>
>どのデータセットもtrain, val, testの3種類に分けて使用してください。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
>Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。
>
>Iris Species
>
>2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。
>
>《ヒント》
>
>以下の2箇所は2クラス分類特有の処理です。
>
>>loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
>>
>>correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
>
>メソッドは以下のように公式ドキュメントを確認してください。
>
>tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow
>
>tf.math.sign  |  TensorFlow
>
>＊tf.signとtf.math.signは同じ働きをします。


In [3]:
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# one-hot-encording
y_one = pd.get_dummies(y)

y = np.array(y_one)
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 10
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net_iris(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net_iris(X)

# 目的関数(他クラス分類用にsoftmaxに書き換え)
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果(他クラス分類用にargmaxを使用)
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))#axis = 1

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)# variableの初期化
    for epoch in range(num_epochs):
        # エポックごとにループ np.ceil=小数の切り上げと切り下げ（切り捨て）
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #train_op = AdamOptimizer Adamアルゴリズムによる勾配降下法
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 5.8965, val_loss : 1.5668, acc : 0.167, val_acc : 0.583
Epoch 1, loss : 2.0338, val_loss : 1.1948, acc : 0.333, val_acc : 0.750
Epoch 2, loss : 0.3107, val_loss : 1.0089, acc : 0.833, val_acc : 0.708
Epoch 3, loss : 0.0471, val_loss : 0.7237, acc : 1.000, val_acc : 0.750
Epoch 4, loss : 0.0135, val_loss : 0.2087, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0626, val_loss : 0.1827, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0921, val_loss : 0.3033, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0127, val_loss : 0.1400, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0054, val_loss : 0.1154, acc : 1.000, val_acc : 0.958
Epoch 9, loss : 0.0081, val_loss : 0.1422, acc : 1.000, val_acc : 0.917
test_acc : 1.000


## 【問題4】House Pricesのモデルを作成
>回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。
>
>House Prices: Advanced Regression Techniques
>
>この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。
>
>分類問題と回帰問題の違いを考慮してください。

回帰問題のため、目的関数（損失関数）は平均二乗誤差を使用。また、Accuracyは削除。

In [4]:
# データセットの読み込み
csv_path = "./Kaggle_data/train.csv" # ファイル名（パス）を指定する
df_house = pd.read_csv(csv_path)

# データフレームから条件抽出
X = np.array(df_house[['GrLivArea','YearBuilt']])
y = np.array(df_house['SalePrice'])

# 対数変換
X = np.log(np.array(X))
y = np.log(np.array(y))
y=y.reshape(-1,1)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 32
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.losses.mean_squared_error(Y, logits)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)# variableの初期化
    for epoch in range(num_epochs):
        # エポックごとにループ np.ceil=小数の切り上げと切り下げ（切り捨て）
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        #total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #train_op = AdamOptimizer Adamアルゴリズムによる勾配降下法
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {}, val_loss : {}".format(epoch, loss, val_loss))
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_los : {:.3f}".format(test_loss))

Epoch 0, loss : 2381.82421875, val_loss : 2374.547607421875
Epoch 1, loss : 14.719456672668457, val_loss : 12.750235557556152
Epoch 2, loss : 1.2193175554275513, val_loss : 1.2378456592559814
Epoch 3, loss : 1.0543290376663208, val_loss : 1.4904065132141113
Epoch 4, loss : 1.0660101175308228, val_loss : 1.0589519739151
Epoch 5, loss : 0.9826500415802002, val_loss : 0.9672972559928894
Epoch 6, loss : 0.8791006207466125, val_loss : 0.8701321482658386
Epoch 7, loss : 0.7745770812034607, val_loss : 0.7751280665397644
Epoch 8, loss : 0.6654304265975952, val_loss : 0.6830049157142639
Epoch 9, loss : 0.5649367570877075, val_loss : 0.599179208278656
test_los : 0.587


## 【問題5】MNISTのモデルを作成  
>ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。
>
>3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。
>
>スクラッチで実装したモデルの再現を目指してください。

#### データセットの用意

In [5]:
import tensorflow as tf
import keras

from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder


(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("MNISTのシェイプ")
print(X_train.shape) # (48000, 784)
print(X_test.shape) # (12000, 784)
print(y_train.shape)
print(y_test.shape)

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

print("平準化後のシェイプ")
print(X_train.shape) # (48000, 784)
print(X_test.shape) # (12000, 784)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

print("ワンホットエンコーディング後のシェイプ")
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

print("train_test_split後のシェイプ")
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)
print(y_train.shape)
print(y_val.shape)

#y_test_decode = np.argmax(y_val, axis=1)
#print(y_test_decode.shape)
#print(y_test_decode)

MNISTのシェイプ
(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)
平準化後のシェイプ
(60000, 784)
(10000, 784)
ワンホットエンコーディング後のシェイプ
(60000,)
(60000, 10)
train_test_split後のシェイプ
(48000, 784)
(12000, 784)
(48000, 10)
(12000, 10)


更新処理をAdaGradに変更。

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 32
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数（Softmaxに変更）
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法(AdaGradに変更)
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果（argmaxに変更）
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))#axis = 1

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)# variableの初期化
    for epoch in range(num_epochs):
        # エポックごとにループ np.ceil=小数の切り上げと切り下げ（切り捨て）
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #train_op = AdamOptimizer Adamアルゴリズムによる勾配降下法
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 0, loss : 33.8318, val_loss : 20.7255, acc : 0.688, val_acc : 0.700
Epoch 1, loss : 28.1194, val_loss : 15.3943, acc : 0.719, val_acc : 0.745
Epoch 2, loss : 24.9740, val_loss : 12.9988, acc : 0.750, val_acc : 0.764
Epoch 3, loss : 23.0304, val_loss : 11.5348, acc : 0.781, val_acc : 0.779
Epoch 4, loss : 21.9172, val_loss : 10.5412, acc : 0.781, val_acc : 0.789
Epoch 5, loss : 20.6765, val_loss : 9.7996, acc : 0.781, val_acc : 0.795
Epoch 6, loss : 19.5364, val_loss : 9.2102, acc : 0.781, val_acc : 0.800
Epoch 7, loss : 18.5220, val_loss : 8.7432, acc : 0.781, val_acc : 0.805
Epoch 8, loss : 17.8902, val_loss : 8.3501, acc : 0.812, val_acc : 0.811
Epoch 9, loss : 17.3614, val_loss : 8.0181, acc : 0.812, val_acc : 0.814
test_acc : 0.824


以上